In [1]:
%cd /content/drive/MyDrive/mushrooms-classification

/content/drive/.shortcut-targets-by-id/1fbjscUK86wUjEst_7siU-DQMq0J3RGUo/mushrooms-classification


In [2]:
from Config import cfg
from Dataset_Generator import get_loaders
from train_utils import update_pretrained_model
from train_ import train_
from eval_  import eval_
import torch.nn as nn
import torch
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings("ignore")

In [6]:

train_loader, val_loader, test_loader = get_loaders(cfg.model.train,
                                                    cfg.model.val,
                                                    cfg.model.test,
                                                    cfg.model.classes,
                                                    cfg.model.batch_size,
                                                    cfg.model.shuffle
                                                    )

In [8]:
model, grad_cam, criterion, optimizer, scheduler, early_stopping, history = \
update_pretrained_model(cfg.model.pretrained_models[2],cfg.model.last_ckpt_path[2])

Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to /root/.cache/torch/hub/checkpoints/convnext_base-6075fbad.pth
100%|██████████| 338M/338M [00:02<00:00, 137MB/s]


In [ ]:
train_(
       train_loader   = train_loader ,
       val_loader     = val_loader ,
       test_loader    = test_loader ,
       model          = model ,
       grad_cam       = grad_cam ,
       optimizer      = optimizer ,
       epochs         = cfg.model.epochs ,
       criterion      = criterion ,
       scheduler      = scheduler ,
       device         = cfg.model.device ,
       last_ckpt_path = cfg.model.last_ckpt_path[2] ,
       best_ckpt_path = cfg.model.best_ckpt_path[2] ,
       early_stopping = early_stopping,
       history        = history
       )

In [ ]:
import numpy as np
all_labels = []
for _, labels in train_loader:
  all_labels.append(labels)
all_labels = np.concatenate(all_labels, 0)

In [ ]:
class_weights =  class_weight.compute_class_weight('balanced',
                                                 classes = np.arange(0,len(cfg.model.classes)),
                                                 y = all_labels)

In [ ]:
class_weights = torch.from_numpy(class_weights).to('cuda',dtype=torch.float)

In [ ]:
criterion = nn.CrossEntropyLoss(weight = class_weights,label_smoothing = 0.1)

In [ ]:
train_(
       train_loader   = train_loader ,
       val_loader     = val_loader ,
       test_loader    = test_loader ,
       model          = model ,
       grad_cam       = grad_cam ,
       optimizer      = optimizer ,
       epochs         = cfg.model.epochs ,
       criterion      = criterion ,
       scheduler      = scheduler ,
       device         = cfg.model.device ,
       last_ckpt_path = cfg.model.last_ckpt_path[2] ,
       best_ckpt_path = cfg.model.best_ckpt_path[2] ,
       early_stopping = early_stopping,
       history        = history
       )

In [ ]:
model, _, criterion, _, _, _, history = \
update_pretrained_model(cfg.model.pretrained_models[2],cfg.model.best_ckpt_path[2])

In [10]:
train_loss, train_accuracy, train_recall_precision = eval_(train_loader, model, criterion, cfg.model.device)
val_loss  , val_accuracy  ,   val_recall_precision = eval_(val_loader, model, criterion, cfg.model.device)
test_loss , test_accuracy ,  test_recall_precision = eval_(test_loader, model, criterion, cfg.model.device)

In [12]:
print(f'train_loss = {round(train_loss / len(train_loader) , 3)}, train_accuracy = {round(train_accuracy / len(train_loader) , 3)}\
    train_recall_precision ={round(train_recall_precision / len(train_loader) , 3)} ')

train_loss = 0.16, train_accuracy = 0.998    train_recall_precision =0.998 


In [13]:
print(f'val_loss = {round(val_loss / len(val_loader), 3)}, val_accuracy = {round(val_accuracy / len(val_loader), 3)}\
      val_recall_precision ={round(val_recall_precision / len(val_loader), 3)} ')

val_loss = 0.419, val_accuracy = 0.914      val_recall_precision =0.912 


In [11]:
print(f'test_loss = {round(test_loss / len(test_loader), 3)}, test_accuracy = {round(test_accuracy / len(test_loader), 3)}\
      test_recall_precision ={round(test_recall_precision / len(test_loader), 3)} ')

test_loss = 0.403, test_accuracy = 0.921      test_recall_precision =0.918 
